In [1]:
import h5py
import pandas as pd
from policyengine_uk import Microsimulation

with h5py.File("constituencies/weights.h5", "r") as f:
    weights = f["weight"][:]
constituencies = pd.read_csv("constituencies/constituencies_2010.csv")

In [2]:
baseline = Microsimulation()

hnet_24 = baseline.calculate("real_household_net_income", 2024)
hnet_29 = baseline.calculate("real_household_net_income", 2029)

In [3]:
change = hnet_29 - hnet_24

In [ ]:
import plotly.express as px
import numpy as np

hex_con = pd.read_csv("constituencies/hex_map/hex_map_2010.csv")

weighted_gain = np.dot(weights, change)
weighted_original_income = np.dot(weights, hnet_24)
weighted_rel_gain = weighted_gain / weighted_original_income

df = pd.DataFrame()
df["Average real change (%)"] = weighted_rel_gain * 100
df["Constituency"] = constituencies.name
df["Code"] = constituencies.code

df["y"] = hex_con.set_index("code").loc[df.Code].y.values
df["x"] = hex_con.set_index("code").loc[df.Code].x.values
df.x = df.x + (df.y % 2 == 0) * -0.5

In [8]:
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="Average real change (%)",
    hover_data=["Constituency"],
)

from policyengine_core.charts import *

fig.update_layout(
    height=580,
    width=700,
)
# Set zero to white
# fig.update_coloraxes(cmax=-df["Average real change"].min(), cmin=df["Average real change"].min())

# Set marker to hexagon
fig.update_traces(marker=dict(symbol="hexagon", size=11))

fig.update_layout(
    title="Baseline forecast for real household net income growth (2024-29)",
    xaxis_tickvals=[],
    xaxis_title="",
    yaxis_tickvals=[],
    yaxis_title="",
    # No bg
    plot_bgcolor="rgba(0,0,0,0)",
    legend_title="Average household net income change",
    # Set color axis to %
    title_font_color="black",
    legend_title_font_color="black",
)

In [ ]:
df[["Code", "Constituency", "Average real change (%)"]].to_csv(
    "constituency_baseline_forecast.csv", index=False
)

In [ ]:
df["Average real change (%)"].describe()

count    650.000000
mean       1.584889
std        0.546521
min        0.344866
25%        1.230439
50%        1.513734
75%        1.941821
max        3.496798
Name: Average real change (%), dtype: float64

In [ ]:
df.sort_values("Average real change (%)")

,Average real change (%),Constituency,Code,y,x
404,0.344866,Sheffield Central,E14001467,-25,58.0
258,0.483102,Leeds North East,E14001321,-19,59.0
550,0.484594,Foyle,N05000008,-15,42.0
217,0.492653,Henley and Thame,E14001280,-35,58.0
114,0.506047,Colne Valley,E14001177,-23,55.0
...,...,...,...,...,...
300,3.236471,Mid Dorset and North Poole,E14001363,-43,50.0
166,3.315424,Erith and Thamesmead,E14001229,-40,66.5
33,3.385172,Birmingham Ladywood,E14001096,-32,53.5
621,3.389432,Blaenau Gwent and Rhymney,W07000084,-33,49.0
